# Comparison of Sentinel-1 RTC products from different software

This notebook was produced as part of the Digital Earth Antarctica (DEAnt) evaluation of different software options to produce SAR RTC data. Four options have been compares using 'on-the-fly' (otf) pipelines developed for each software. The four softwares options of interest are:

In [ ]:
import os
import h5py
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import boto3
import pandas as pd
import json
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
import rioxarray
import asf_search as asf
from shapely.geometry import Polygon
from celluloid import Camera # getting the camera
from IPython.display import HTML

sns.set_theme()

%matplotlib inline

# Functions

In [ ]:
def make_gif(imgs, vmin, vmax, title=''):
    fig, ax = plt.subplots() # make it bigger
    camera = Camera(fig)# the camera gets our figure
    for i,img in enumerate(imgs):
        im = ax.imshow(img,
                  vmin=vmin,
                  vmax=vmax) # plotting
        ax.set_title(f'{title}')
        camera.snap()
    animation = camera.animate()
    return animation

def assign_crs(tif_path, crs):
    with rasterio.open(tif_path, 'r+') as rds:
        rds.crs = CRS.from_epsg(crs)

def plot_tifs(
    tif_1, 
    tif_2, 
    titles= ['arr_1', 'arr_2'],
    suptitle='suptitle',
    convert_dB = True,
    scale=[-40,10],
    cmap = 'binary_r',
    save_path=''):

    # place to store data
    hist_data, crss, meta = [],[], []
    colors = ['red', 'blue']

    # plot the tif
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(18,10))
    for i, tif in enumerate([tif_1, tif_2]):
        with rasterio.open(tif.local_file) as src:
                data = src.read(1)
                nodata = src.nodata
                # covert from linear to db
                if convert_dB:
                    data = 10*np.log10(data)
                # covert no data to nan
                data[data==nodata] = np.nan
                crss.append(src.meta['crs'])
                im = ax[i].imshow(data, cmap=cmap, vmin=scale[0], vmax=scale[1])
                ax[i].set_title(f'{titles[i]}')
                hist_data.append(data[(np.isfinite(data))])
                meta.append(src.meta.copy())

    plt.suptitle(f'{suptitle}', y=0.9)
    cbar_ax = f.add_axes([0.95, 0.15, 0.04, 0.7])
    f.colorbar(im, cax=cbar_ax)
    plt.show()

    # plot the histogram 
    for i in [0,1]:
        u, std = np.mean(hist_data[i]), np.std(hist_data[i])
        plt.hist(hist_data[i], 
                density=True,
                bins=60, 
                alpha=0.5, 
                label=f'{titles[i]}; u={u:.3f}, std={std:.3f}', 
                color=colors[i],
                histtype='step')

    plt.title(f'{suptitle}')
    plt.xlabel('Gamma0 RTC')
    plt.ylabel('Frequency')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

    for i, tif in enumerate([tif_1, tif_2]):
        print(tif)
        for k in meta[i].keys():
            print(f'{k} : {meta[i][k]}')
        print('\n')

    if save_path:
        plt.savefig(save_path)

    
def plot_difference_maps(
    arr_1, 
    arr_2, 
    titles=['arr_1','arr_2','diff (arr_1 - arr_2)'],
    scales = [[-40,10],[-40,10],[-1,1]],
    ylabels=['decibels (dB)','decibels (dB)','decibels (dB)'],
    save_path=''):
    
    diff = arr_1 - arr_2
    arrs = [arr_1, arr_2, diff]
    stats_arr = np.array(diff)[np.array((np.isfinite(diff)))]
    print('Difference Stats')
    print(f'min: {stats_arr.min()}', 
        f'max: {stats_arr.max()}',
        f'mean: {stats_arr.mean()}',
        f'median: {np.percentile(stats_arr, 50)}',
        f'5th percentile: {np.percentile(stats_arr, 5)}',
        f'90th percentile: {np.percentile(stats_arr, 95)}',
        )

    cmaps = ['binary_r','binary_r','bwr']

    f, ax = plt.subplots(nrows=4, ncols=1, figsize=(10,40))
    for i,arr in enumerate(arrs):
        im = ax[i].imshow(arr[0], 
                vmin = scales[i][0], 
                vmax = scales[i][1],
                cmap = cmaps[i])
        ax[i].set_title(titles[i])
        f.colorbar(im, ax=ax[i], label=ylabels[i])
        
    # plot the histogram
    colors = ['red','blue']
    for i in [0,1]:
        # only get real values 
        hist_data = np.array(arrs[i])[
                (np.isfinite(np.array(arrs[i])))
                ]
        u, std = np.mean(hist_data), np.std(hist_data)
        ax[3].hist(hist_data, 
                density=True,
                bins=60, 
                alpha=0.5, 
                label=f'{titles[i]}; u={u:.3f}, std={std:.3f}', 
                color=colors[i],
                histtype='step')
        ax[3].set_title('Pixel distribution')

    plt.legend(loc='best')
    if save_path:
        plt.savefig(save_path)


# Settings

In [ ]:
# general structure for scenes in s3
# s3_bucket/software/dem/scene/scene_files
s3_bucket = 'deant-data-public-dev'
s3_bucket_link = 'https://deant-data-public-dev.s3.ap-southeast-2.amazonaws.com/'

scenes = [
        'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63',
        'S1A_IW_SLC__1SSH_20190605T222724_20190605T222751_027550_031BE1_AD3A',
        'S1A_IW_SLC__1SSH_20190926T124734_20190926T124804_029192_0350B9_FA6B',
        'S1A_IW_SLC__1SSH_20230127T142750_20230127T142817_046970_05A22F_17F7',
        'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B',
        'S1B_IW_SLC__1SSH_20210223T233056_20210223T233124_025740_031194_E7BE',
        'S1B_IW_SLC__1SSH_20210228T035005_20210228T035033_025801_03138F_8CB2',
]
softwares = ['pyrosar','rtc-opera','hyp3-gamma', 'S1_NRB']
dems = ['glo_30','REMA_32']

# get crededentials for AWS
with open('aws_credentials.txt') as f:
    ACCESS_ID, ACCESS_KEY = f.readlines()
    ACCESS_ID = ACCESS_ID.strip()
    ACCESS_KEY = ACCESS_KEY.strip()

# setup s3
s3 = boto3.client('s3', 
                        region_name='ap-southeast-2',
                        aws_access_key_id=ACCESS_ID,
                        aws_secret_access_key= ACCESS_KEY)

# make data directory to store local files
os.makedirs('data', exist_ok=True)

## Show example scene files for software

**Products**
- RTC products are stored in the s3 bucket specified above
- The general pattern for the products is : ```software/dem/crs/scene/files```

**Projections**
- pyrosar and rtc-opera are projected in 3031 polar steregraphic coordinates 
- hype-gamma and S1_NRB products are provided in UTM zone projections (output crs cannot be specified)
- For a fair comparison, products should be compared in the same projection

**Data**
- Naming convention for rtc backscatter differs for each product

**Metadata**
- Metadata products are stored differently between products

In [ ]:
file_list = []
for software in softwares:
    for dem in dems:
        params = {
            "Bucket": f'{s3_bucket}',
            "Prefix": f'{software}/{dem}'
        }
        objects = s3.list_objects_v2(**params)
        if 'Contents' in objects.keys():
            data = objects['Contents']
            file_list.extend([x for x in objects['Contents']])

# save all of the files in a dataframe for east of searching
df_s3 = pd.DataFrame.from_records(file_list)
df_s3[['software','dem','crs','scene','file']] = df_s3['Key'].str.split('/', n=4, expand=True)
df_s3.head(2)

# Compare total timing

In [ ]:
# limit to single dem and proj
df_timing_files = df_s3[(
    (df_s3['file'].str.contains('_timing.json'))
    #& (df_s3['dem']==dem)
    #& (df_s3['crs']==proj
    )]
df_timing_files.head(2)

In [ ]:
timing_data = []
for i in range(0,len(df_timing_files)):
    timing_file = df_timing_files.iloc[i].Key
    print(timing_file)
    try:
        s3.download_file(s3_bucket, timing_file, 'tmp.json')
        with open('tmp.json') as json_file:
            data = json.load(json_file)
            data['software'] = df_timing_files.iloc[i].software
            data['scene'] = df_timing_files.iloc[i].scene
        timing_data.append(data)
        print(f'downloaded: {timing_file}')
    except:
        print(f'no timing file: {timing_file}')

os.remove('tmp.json')
df_timing = pd.DataFrame.from_records(timing_data, index=['software','scene'])

# gamma values are in list for some reason
for col in list(df_timing):
    df_timing[col] = df_timing[col].apply(lambda x : x[0] if isinstance(x,list) else x)

# min timing filter to remove false information
min_time = 500
df_timing = df_timing[df_timing['Total']>min_time]

# plot mean time by software
sw_count = df_timing.groupby('software').size()
ax = (df_timing.groupby('software').mean()
 .drop(columns=['Total'])
 .plot.bar(stacked=True))
ax.set_xlabel('Software')
ax.set_ylabel('Time (seconds)')
ax.set_title(f'Software Processing Mean Times (DEM upsampling=2)')
print(sw_count)

# Compare Scenes
**Differences**
- Treatment of burst overlaps

## Conditions for the comparison
- we want a single dependant variable to compare the scenes

In [ ]:
#df_s3.groupby(['scene','software'])['crs'].unique()
df_s3

In [ ]:
# make the settings for the comparison
# compare 1 scene agains a single dependant variable (e.g. dem, software, proj)
scene = 'S1A_IW_SLC__1SSH_20230116T100627_20230116T100655_046807_059CB3_FCC7'
dependant_var = 'software'
dependant_vals = ['pyrosar','hyp3-gamma']
independant_var1 = 'dem'
independant_val1 = 'glo_30'
target_crs = 3031

print(f'Comparing scenes with varying {dependant_var} : {dependant_vals}')
print(f'Keeping {independant_var1} fixed: {independant_val1}')

df_comparison = df_s3[(
    (df_s3[dependant_var].isin(dependant_vals))
    & (df_s3[independant_var1] == independant_val1)
    & (df_s3['scene'] == scene)
)]

scene_tifs = df_comparison[(
      (df_comparison.file.str.contains('rtc|HH')) &
      (df_comparison.file.str.endswith('tif'))
      )]
scene_dems = df_comparison[(df_comparison.file.str.endswith('_dem.tif'))]
print(f'{len(scene_tifs)} scene tifs found meeting conditions')
print(f'{len(scene_dems)} scene dems found meeting conditions')
assert len(scene_tifs)==2, 'just two scenes meeting conditions are required for comparison'
assert len(scene_tifs)==2, 'just two dems meeting conditions are required for comparison'

# determine which scenes need to be reprojected
scene_tifs['reproject'] = scene_tifs['crs'].apply(lambda x : False if str(x) == str(target_crs) else True)
scene_tifs[['software','dem','crs','Size','reproject','file']]

## Download the tifs

In [ ]:
# download tifs and store locally
download = False
if download:
    for i in range(0,len(scene_tifs)):
        key = scene_tifs.iloc[i].Key
        filename = scene_tifs.iloc[i].file
        print(f'downloading {filename}')
        s3.download_file(s3_bucket, key, f'data/tifs/{filename}')
scene_tifs['local_file'] = scene_tifs['file'].apply(lambda x : f'data/tifs/{x}')

## Comparison of raw outputs

In [ ]:
# plot the two tiffs side by side with native projections, shapes etc
tif_1_data = scene_tifs.iloc[0]
tif_2_data = scene_tifs.iloc[1]

plot_tifs(
    tif_1_data, 
    tif_2_data, 
    titles= dependant_vals,
    suptitle=f'{scene}',
    convert_dB = True,
    cmap = 'binary_r',
    scale=[-40,10])


## Raster Difference Maps
- Project rasters to the same resolutions and shapes to enable a direct difference
- Note differences may be due to geometric differences and not intensity
- Be sure to inspect the pixel shift below

In [ ]:
# get the scene geom in 4326
asf.constants.CMR_TIMEOUT = 45
asf_result = asf.granule_search([scene], asf.ASFSearchOptions(processingLevel='SLC'))[0]
points = (asf_result.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
                ['Geometry']['GPolygons'][0]['Boundary']['Points'])
points = [(p['Longitude'],p['Latitude']) for p in points]
scene_poly = Polygon(points)
str(scene_poly)

In [ ]:
# local files
local_tif_1_path = tif_1_data.local_file
local_tif_2_path = tif_2_data.local_file
#assign_crs(local_tif_1_path, 3031) # assign missing crs to pyrosar...
print(local_tif_1_path)
print(local_tif_2_path)
tif_1_f = rioxarray.open_rasterio(local_tif_1_path)
tif_2_f = rioxarray.open_rasterio(local_tif_2_path)
# clip by the scene geometry
tif_1_clipped = tif_1_f.rio.clip([scene_poly], CRS.from_epsg(4326))
tif_2_clipped = tif_2_f.rio.clip([scene_poly], CRS.from_epsg(4326))
del tif_1_f
del tif_2_f
print(tif_1_clipped.shape, tif_2_clipped.shape)
# match the projection/transform/shape
tif_1_matched = tif_1_clipped.rio.reproject_match(tif_2_clipped)
print(tif_1_matched.shape)
# convert to db
tif_1_db = 10*np.log10(tif_1_matched)
tif_2_db = 10*np.log10(tif_2_clipped)
# calculate the difference between the two images
diff = tif_1_db - tif_2_db
# relative difference as a % of tif_2
rel_deff = 100*(diff/tif_2_clipped)
# save tifs
tif_1_db.rio.to_raster(f'data/tifs/{scene}_{dependant_vals[0]}_clipped.tif')
tif_2_db.rio.to_raster(f'data/tifs/{scene}_{dependant_vals[1]}_clipped.tif')
diff.rio.to_raster(f'data/tifs/{scene}_diff_clipped.tif')

In [ ]:
# resample 
# upscale_factor = 0.1
upscale_factor = False
if upscale_factor:
    new_width = int(tif_1_db.rio.width * upscale_factor)
    new_height = int(tif_1_db.rio.height * upscale_factor)

    tif_1_db = tif_1_db.rio.reproject(
        tif_1_db.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
    )

    tif_2_db = tif_2_db.rio.reproject(
        tif_2_db.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
    )

    diff = tif_1_db - tif_2_db
    print(diff.shape)

In [ ]:
scales = [[-40,10],[-40,10],[-1,1]]
titles = [f'{dependant_vals[0]}',
          f'{dependant_vals[1]}',
          f'abs difference ({dependant_vals[0]} - {dependant_vals[1]}-rtc)']
plot_difference_maps(
      tif_1_db, 
      tif_2_db,
      titles=titles,
      scales=scales,
      save_path=f'data/compare-rtc/{scene}_{dependant_vals[0]}_vs_{dependant_vals[1]}.png')

# See pixel shift in small area

In [ ]:
#x1,x2,y1,y2 = 8600,9000,6600,7000 # full res
#x1,x2,y1,y2 = 3400,4000,9400,10000 # full res
x1,x2,y1,y2 = 6500,6900,4600,5000 # full res
#x1,x2,y1,y2 = 2000,2500,8600,9100 # full res
#x1,x2,y1,y2 = 7000,7500,3800,4300# full res
if upscale_factor:
    x1,x2,y1,y2 = [int(n*upscale_factor) for n in [x1,x2,y1,y2]] # adjust for scaling
tif_1_snip = tif_1_db[0][y1:y2,x1:x2]
tif_2_snip = tif_2_db[0][y1:y2,x1:x2]
animation = make_gif(
    [tif_2_snip, tif_1_snip], 
    vmin=-40, 
    vmax=10, 
    title=f'{dependant_vals[0]}_vs_{dependant_vals[1]}')
animation.save(f'data/compare-rtc/{scene}_{dependant_vals[0]}_vs_{dependant_vals[1]}.gif')
HTML(animation.to_html5_video())

# Corestister the Images with Arosics

In [ ]:
import arosics
# img_trg is the one that is shifted
coreg = arosics.CoReg.COREG(
    im_ref=f'data/tifs/{scene}_{dependant_vals[1]}_clipped.tif', 
    im_tgt=f'data/tifs/{scene}_{dependant_vals[0]}_clipped.tif', 
    path_out=f'data/tifs/{scene}_{dependant_vals[0]}_clipped_aligned.tif', 
    fmt_out='GTIFF',
    align_grids=True)
res = coreg.correct_shifts()

In [ ]:
tif_1_db = rioxarray.open_rasterio(f'data/tifs/{scene}_{dependant_vals[0]}_clipped_aligned.tif')
tif_2_db = rioxarray.open_rasterio(f'data/tifs/{scene}_{dependant_vals[1]}_clipped.tif')

tif_1_snip = tif_1_db[0][y1:y2,x1:x2]
tif_2_snip = tif_2_db[0][y1:y2,x1:x2]
animation = make_gif(
    [tif_2_snip, 
    tif_1_snip], 
    vmin=-40, 
    vmax=10, 
    title=f'{dependant_vals[0]}_vs_{dependant_vals[1]}_aligned')
animation.save(f'data/compare-rtc/{scene}_{dependant_vals[0]}_vs_{dependant_vals[1]}_aligned.gif')
HTML(animation.to_html5_video())

In [ ]:
scales = [[-40,10],[-40,10],[-1,1]]
titles = [f'{dependant_vals[0]}',
          f'{dependant_vals[1]}',
          f'abs difference ({dependant_vals[0]} - {dependant_vals[1]})']
plot_difference_maps(
      tif_1_db, 
      tif_2_db,
      titles=titles,
      scales=scales,
      save_path=f'data/compare-rtc/{scene}_{dependant_vals[0]}_vs_{dependant_vals[1]}_aligned.png')

# Compare the DEMS

In [ ]:
# download tifs and store locally
local_dems = []
download = False
if download:
      for i in range(0,len(scene_dems)):
            key = scene_dems.iloc[i].Key
            filename = scene_dems.iloc[i].file
            sw = scene_dems.iloc[i].software
            s3.download_file(s3_bucket, key, f'data/tifs/{sw}_{filename}')
            local_dems.append(f'data/tifs/{sw}_{filename}')
local_dems

In [ ]:
# clear mem
tif_1_db = tif_2_db = coreg = None

In [ ]:
# load in the dems
tif_1_dem = rioxarray.open_rasterio(f'data/tif_1_{scene}_dem.tif')
tif_2_dem = rioxarray.open_rasterio(f'data/rtc-tif_2_{scene}_dem.tif')
# clip by scene geom
tif_1_dem = tif_1_dem.rio.clip([scene_poly], CRS.from_epsg(4326))
tif_2_dem = tif_2_dem.rio.clip([scene_poly], CRS.from_epsg(4326))
print(tif_1_dem.shape, tif_2_dem.shape)
# match the projection/transform/shape of the dems
tif_1_dem = tif_1_dem.rio.reproject_match(tif_2_dem)
print(tif_1_dem.shape, tif_2_dem.shape)

# convert to np arrays
tif_1_dem = np.array(tif_1_dem)
tif_2_dem = np.array(tif_2_dem)
tif_1_dem[(tif_1_dem==-9999)] = np.nan # replace nodata with -9999

In [ ]:
# try and adjust for pixel size (30m/20)
x1,x2,y1,y2 = (np.array((x1,x2,y1,y2))*(2/3)).astype(int)
pyrosar_snip = pyrosar_dem[0][y1:y2,x1:x2]
opera_snip = opera_dem[0][y1:y2,x1:x2]
# replace pyrosar nodata of -9999 with nans
animation = make_gif([opera_snip, pyrosar_snip], vmin=None, vmax=None)
HTML(animation.to_html5_video())

In [ ]:
scales = [[None,None],[None,None],[None,None]]
titles = ['pyrosar dem (glo-30)',
          'opera-rtc dem (glo-30)',
          'abs difference (pyrosar - opera-rtc)']
ylabels = ['elevation (m)', 'elevation (m)', 'difference (m)']
plot_difference_maps(
      pyrosar_dem, 
      opera_dem,
      titles=titles,
      scales=scales,
      ylabels=ylabels
      )

# Scratch

In [ ]:
def generate_extra_points_along_boundary(bbox, delta=0.1):
    """
    Generate points along the boundary of a bounding box.

    Parameters:
    - bbox: Tuple of four coordinates (x_min, y_min, x_max, y_max).
    - delta: distance between points along the bounding box sides 

    Returns:
    - List of points [(x1, y1), (x2, y2), ...] along the boundary.
    """
    x_min, y_min, x_max, y_max = bbox
    # Generate points along the top side
    top_side = [(x, y_max) for x in list(np.arange(x_min, x_max, delta)) + [x_max]]    
    # Generate points along the right side
    right_side = [(x_max, y) for y in list(np.arange(y_max - delta, y_min-delta, -delta)) + [y_min-delta]]
    # Generate points along the bottom side
    bottom_side = [(x, y_min) for x in list(np.arange(x_max - delta, x_min-delta, -delta)) + [x_min-delta]]
    list(np.arange(y_min + delta, y_max, delta)) + [y_max]
    # Generate points along the left side
    left_side = [(x_min, y) for y in list(np.arange(y_min + delta, y_max, delta)) + [y_max]]
    # Combine all sides' points
    all_points = top_side + right_side + bottom_side + left_side
    return all_points

# Example usage:
bounding_box = (150, -75, 160, -70)
points_along_boundary = generate_points_along_boundary(bounding_box)
print(points_along_boundary)
Polygon(points_along_boundary)
